In [1]:
%pylab inline
from qualityBaseline import *
from analysis_toolbox import *

Populating the interactive namespace from numpy and matplotlib


qualityBaseline.py:219: SyntaxWarning: import * only allowed at module level
  def compute_my_COCO_BBvals(compute_metrics=['simple','area','dist']):


#### Variable defintions

|Math notation  | Variable name | Meaning|
| ------------- |:-------------:| -----|
|$\mathcal{I}$ | I|set of all images seen |
|$\mathcal{U}$ | U|set of all images |
|$\mathcal{E}$ | E|expert list|
|$\mathcal{B}$ | B|'bot' (bad workers) list|
|$p(l_{ij}|z_i)$ | plij|posterior|
|$p(a_j)$ | pa| prob of honest worker|

In [133]:
img_info,object_tbl,bb_info,hit_info = load_info()
ai_tbl= pd.read_csv("interpolated_aligned_bb_info.csv",index_col=0)

In [145]:
U = list(img_info.id)

In [141]:
I = []
L = []
E = []
B = []
U=[]
while len(I)<len(U): 
    # add a image in U but not in I (pick randomly)
    I.append(random.choice([u for u in U if u not in I ]))
    # Looping through all the seen images 
    for i in I: 
    